In [1]:
import pandas as pd
import numpy as np
import datetime
import math
import jieba
import jieba.posseg as psg
from gensim import corpora, models
from jieba import analyse
import functools
import re
from pyltp import SentenceSplitter


In [2]:
all_docs_df = pd.read_csv('../../data/chusai/all_docs.txt', sep='\001', header=None)
all_docs_df.columns = ['id', 'title', 'text']
print(all_docs_df.info())
print(all_docs_df.head())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 108295 entries, 0 to 108294
Data columns (total 3 columns):
id       108295 non-null object
title    108294 non-null object
text     108293 non-null object
dtypes: object(3)
memory usage: 2.5+ MB
None
        id                title  \
0  D000001   林志颖老婆深夜敷面膜，睫毛太长好吓人   
1  D000002   小s夸杨幂身材好，杨幂回复太精彩了！   
2  D000003    年轻时的她风华绝代，现却无人送祝福   
3  D000004   林心如屡曝霍建华私生活被怼蹭老公人气   
4  D000005  曾是TVB颜值担当，近照曝光发现真老了   

                                                text  
0  早年林志颖带kimi上《爸爸去哪儿》的时候，当时遮遮掩掩的林志颖老婆低调探班，总让人觉得格外...  
1  翩若惊鸿，婉若游龙。曹植形容洛神的这两句，实在太抽象，以至于始终寻不到承受对象。直到在《大军...  
2  上个世纪香港影视界涌现出了不少高颜值女星，在《大话西游之月光宝盒》中饰演春三十娘和蜘蛛精的蓝...  
3  霍建华林心如1905电影网讯近日，林心如在接受采访时爆料称老公霍建华会主动向女儿索吻，笑称他...  
4  不知道有多少人是看TVB剧集长大的，小时候我每一天晚上都会守着电视看TVB剧集的。可以说对于...  


In [3]:
train_doc_keyword_df = pd.read_csv('../../data/chusai/train_docs_keywords.txt', sep='\t', header=None)
train_doc_keyword_df.columns = ['id', 'keyword']
train_doc_keyword_df['keyword_list'] = train_doc_keyword_df['keyword'].map(lambda x: x.split(','))
train_doc_keyword_df = pd.merge(train_doc_keyword_df, all_docs_df, on='id', how='left')
print(train_doc_keyword_df.head(10))


        id            keyword           keyword_list  \
0  D012650         三生三世,凤九,东华         [三生三世, 凤九, 东华]   
1  D047118              南宁,美食               [南宁, 美食]   
2  D098970             凉菜,北京人              [凉菜, 北京人]   
3  D092010          华为,P30pro           [华为, P30pro]   
4  D103408             酒吧,世界杯              [酒吧, 世界杯]   
5  D103809            烧烤,人生一串             [烧烤, 人生一串]   
6  D003823         惠若琪,杨颖,奔跑吧         [惠若琪, 杨颖, 奔跑吧]   
7  D023499  我与你的光年距离2,王以纶,许晓诺  [我与你的光年距离2, 王以纶, 许晓诺]   
8  D063937          无人机,军用,美军          [无人机, 军用, 美军]   
9  D059085           女兵,剪影,欧美           [女兵, 剪影, 欧美]   

                                    title  \
0          《三生三世》里凤九东华最有“夫妻相”，这一模一样的动作是证明   
1        【味集】南宁美食圣地最强攻略！老南宁最爱的味道，都汇集在这些地方   
2                  据说这些下酒凉菜，只有北京人喜欢，是这样吗？   
3  华为新机皇P30pro曝光：麒麟1020＋5G芯片，网友：小米要拿什么来抵挡   
4                  想在泰晤士河边餐厅酒吧看世界杯？戳进来就对啦   
5   看完真的会饿!有人把【烧烤】拍成了纪录片,怒摘9.8高分,市井烟火气太浓~   
6          杨颖跳高只离地10公分，被众人无情嘲笑，惠若琪：你打他牙干嘛   
7                 《我与你的

In [8]:
print(train_doc_keyword_df[['keyword', 'title', 'text']][480:510])



                keyword                                title  \
480       乔梁,真爱的谎言之破冰者,            《真爱的谎言之破冰者》靳远套路乔梁，蔡炳坤套路逗逗   
481             魔兽争霸,暴雪         暴雪《魔兽争霸》十五年后重置？孕育MOBA了，启蒙了电竞   
482   手机,小米,华为,P20,荣耀10             一山难容二虎，华为小试牛刀之作，却成小米最大劲敌   
483             刘嘉玲,娱乐圈                 刘嘉玲素颜和男友人外出游玩纤细身材超吸睛   
484        LOL,faker,恩静    LOL：faker和恩静的前世今生恩静要结婚了，那飞科变捞的原因？   
485    肠道炎症模型,斑马鱼,炎症性肠病                广州生物院在斑马鱼中构建出一种肠道炎症模型   
486         烽火线,红楼梦,闵春晓                 闵春晓巾帼不让须眉突破形象出演《烽火线》   
487         抖音,字节跳动,张一鸣                           麻烦不断抖音再被封禁   
488        蜀山之白眉真人传,王铭铎              《蜀山之白眉真人传》杀青王铭铎虐心黑化为爱痴狂   
489     彩虹无人机,高原作战,高原试飞           彩虹无人机集体上高原！打破中国无人机不能高原作战谬论   
490    政变,土耳其,政变,中东,库尔德        土耳其秋后算账，对参与2016年政变人员出重拳，谴责美国！   
491          加勒比香蕉大盗,香蕉         大B哥新作，有意思：“加勒比香蕉大盗”（已打包·可下载）   
492         口腔异味,预防晨起口臭                   口腔异味？这五种食物帮你快速去异味!   
493  生机无限,湖南卫视,国式老人,节约观               湖南卫视《生机无限》直击“中国式老人”节约观   
494      黄奕,黄毅清,黄奕名誉维权案           黄奕黄毅清维

In [19]:
def get_text_sentences(text):
    sentences_list = SentenceSplitter.split(text)
    return sentences_list

all_docs_df['text'] = all_docs_df['text'].map(lambda x: str(x))
all_docs_df['text_sentences'] = all_docs_df['text'].map(lambda x: get_text_sentences(x))
all_docs_df['text_sentences_len'] = all_docs_df['text_sentences'].map(lambda x: len(x))
print(all_docs_df[['text', 'text_sentences', 'text_sentences_len']].head())
    

                                                text  \
0  早年林志颖带kimi上《爸爸去哪儿》的时候，当时遮遮掩掩的林志颖老婆低调探班，总让人觉得格外...   
1  翩若惊鸿，婉若游龙。曹植形容洛神的这两句，实在太抽象，以至于始终寻不到承受对象。直到在《大军...   
2  上个世纪香港影视界涌现出了不少高颜值女星，在《大话西游之月光宝盒》中饰演春三十娘和蜘蛛精的蓝...   
3  霍建华林心如1905电影网讯近日，林心如在接受采访时爆料称老公霍建华会主动向女儿索吻，笑称他...   
4  不知道有多少人是看TVB剧集长大的，小时候我每一天晚上都会守着电视看TVB剧集的。可以说对于...   

                                      text_sentences  text_sentences_len  
0  [早年林志颖带kimi上《爸爸去哪儿》的时候，当时遮遮掩掩的林志颖老婆低调探班，总让人觉得格...                  25  
1  [翩若惊鸿，婉若游龙。, 曹植形容洛神的这两句，实在太抽象，以至于始终寻不到承受对象。, 直...                   8  
2  [上个世纪香港影视界涌现出了不少高颜值女星，在《大话西游之月光宝盒》中饰演春三十娘和蜘蛛精的...                   7  
3  [霍建华林心如1905电影网讯近日，林心如在接受采访时爆料称老公霍建华会主动向女儿索吻，笑称...                   9  
4  [不知道有多少人是看TVB剧集长大的，小时候我每一天晚上都会守着电视看TVB剧集的。, 可以...                  27  


In [20]:
print(np.mean(all_docs_df['text_sentences_len']))


23.561494067131445


In [22]:
def get_title_text(all_docs_df):
    temp_df = pd.DataFrame(columns=['id', 'title_text'])
    for temp_id, title, text, text_sentences_len in all_docs_df[['id', 'title', 'text', 'text_sentences_len']].values:
        length = math.ceil(text_sentences_len / 2)
        title_text = ''
        for i in range(length):
            title_text = title + '。' + title_text
        title_text = title_text + text
        temp = pd.DataFrame([[temp_id, title_text]], columns=['id', 'title_text'])
        temp_df = pd.concat([temp_df, temp])
    all_docs_df = pd.merge(all_docs_df, temp_df, on='id', how='left')
    return all_docs_df

all_docs_df['title'] = all_docs_df['title'].astype(str)
all_docs_df['text'] = all_docs_df['text'].astype(str)
all_docs_df = get_title_text(all_docs_df)
print(all_docs_df[['title', 'title_text']].head())

    
    

                 title                                         title_text
0   林志颖老婆深夜敷面膜，睫毛太长好吓人  林志颖老婆深夜敷面膜，睫毛太长好吓人。林志颖老婆深夜敷面膜，睫毛太长好吓人。林志颖老婆深夜敷...
1   小s夸杨幂身材好，杨幂回复太精彩了！  小s夸杨幂身材好，杨幂回复太精彩了！。小s夸杨幂身材好，杨幂回复太精彩了！。小s夸杨幂身材好...
2    年轻时的她风华绝代，现却无人送祝福  年轻时的她风华绝代，现却无人送祝福。年轻时的她风华绝代，现却无人送祝福。年轻时的她风华绝代，...
3   林心如屡曝霍建华私生活被怼蹭老公人气  林心如屡曝霍建华私生活被怼蹭老公人气。林心如屡曝霍建华私生活被怼蹭老公人气。林心如屡曝霍建华...
4  曾是TVB颜值担当，近照曝光发现真老了  曾是TVB颜值担当，近照曝光发现真老了。曾是TVB颜值担当，近照曝光发现真老了。曾是TVB颜...


In [14]:
def is_contain_alpha(x):
    my_re = re.compile(r'[A-Za-z]',re.S)
    res = re.findall(my_re,x)
    if len(res):
        return True
    else:
        return False

string1 = '123s'
string2 = '应该，步子到，kds123'
print(is_contain_alpha(string1))
print(is_contain_alpha(string2))


True
True
